In [ ]:
import os.path
from os import path
import numpy as np
import matplotlib.pyplot as pyplot
import gc
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
import statistics
import seaborn as sn
import random
import copy
import math
from itertools import permutations

Fill in the following paths that contain:

neutrons_test_path: neutron pulses used for testing\
gammas_test_path: gamma pulses used for testing\
template_n_path: template of the average neutron\
template_g_path: template of the average gamma\
classifier_config_path: constants saved in the classifiers training
path_model_FC_NN: Trained raw pulse DNN\
path_model_FC_NN_params: Trained raw pulse DNN parameters\
path_model_Concat: Trained features DNN\
path_model_Attention: Trained Deep Classifier\
path_model_Attention_params: Trained features DNN + Deep Classifier parameters\
path_model_BM_FCNN: Trained DNN 6 outputs\
path_model_BM_FCNN_params: Trained DNN 6 outputs parameters\
path_model_BM_FCNN1: Trained Two DNNs\
path_model_BM_FCNN1_params: Trained Two DNNs parameters\
path_CI_parameters: Saved CI parameters\
path_NCC_classifier_parameters: Saved NCC parameters\


In [ ]:
neutrons_test = np.load(neutrons_test_path)
gammas_test = np.load(gammas_test_path)
template_n = np.load(template_n_path)
template_g = np.load(template_g_path)

with open(classifier_config_path, "rb") as file:
  constants_dict = pickle.load(file)

arrival_rate = constants_dict["arrival_rate"]
g_n_ratio = constants_dict["g_n_ratio"]
starting_index = constants_dict["starting_index"]
template_length_orig_features = constants_dict["template_length_orig_features"]
Ts = constants_dict["Ts"]
num_original_features = constants_dict["num_original_features"]
num_classification_features = constants_dict["num_classification_features"]
num_attention_features = constants_dict["num_attention_features"]
new_starting_point = constants_dict["new_starting_point"]
pulse_len_after_starting = constants_dict["pulse_len_after_starting"]
time_offset = constants_dict["time_offset"]
success_marg = constants_dict["success_marg"]
L_deriv_orig_features = constants_dict["L_deriv_orig_features"]
delta_orig_features = constants_dict["delta_orig_features"]
short_gate_orig_features = constants_dict["short_gate_orig_features"]
long_gate_orig_features = constants_dict["long_gate_orig_features"]
max_random_offset = constants_dict["max_random_offset"]
cont_starting_time = constants_dict["cont_starting_time"]
original_pulse_len = constants_dict["original_pulse_len"]
std_noise_total = constants_dict["std_noise_total"]
std_noise_curr_constant = constants_dict["std_noise_curr_constant"]
noise_mean = constants_dict["noise_mean"]
len_true_inf = constants_dict["len_true_inf"]
lambda_n = constants_dict["lambda_n"]
lambda_g = constants_dict["lambda_g"]
std_noise_curr = constants_dict["std_noise_curr"]
std_added_noise = constants_dict["std_added_noise"]
arrival_rate_text = constants_dict["arrival_rate_text"]
curr_path = constants_dict["curr_path"]
short_gate_class_features = constants_dict["short_gate_class_features"]
long_gate_class_features = constants_dict["long_gate_class_features"]
start_gate_class_features = constants_dict["start_gate_class_features"]
corr_start_class_features = constants_dict["corr_start_class_features"]
corr_end_class_features = constants_dict["corr_end_class_features"]
Num_train = constants_dict["Num_train"]
Num_val = constants_dict["Num_val"]
Num_test = constants_dict["Num_test"]
Num_BM = constants_dict["Num_BM"]
WL_classifier = constants_dict["WL_classifier"]
model_classifier_layer1 = constants_dict["model_classifier_layer1"]
model_classifier_layer2 = constants_dict["model_classifier_layer2"]
model_classifier_layer3 = constants_dict["model_classifier_layer3"]
model_classifier_layer4 = constants_dict["model_classifier_layer4"]
model_classifier_layer5 = constants_dict["model_classifier_layer5"]
model_attention_layer1 = constants_dict["model_attention_layer1"]
model_attention_layer2 = constants_dict["model_attention_layer2"]
model_FC_NN_lr = constants_dict["model_FC_NN_lr"]
model_FC_NN_bs = constants_dict["model_FC_NN_bs"]
model_FC_NN_epochs = constants_dict["model_FC_NN_epochs"]
model_Attention_lr = constants_dict["model_Attention_lr"]
model_Attention_bs = constants_dict["model_Attention_bs"]
model_Attention_epochs = constants_dict["model_Attention_epochs"]
max_dist = constants_dict["max_dist"] #

run <your-path> 'General_Functions.ipynb>
run <your-path> 'Classification_Algorithm_Functions.ipynb>

# Load the DNN applied to the raw pulse x_t model
model_fc_nn = torch.load(path_model_FC_NN)
norm_params_model_fc_nn = np.load(path_model_FC_NN_params)

# Load the DNN applied to both the raw pulse x_t and classical features y_t model
model_concat = torch.load(path_model_Concat)
norm_params_model_concat = np.load(path_model_Attention_params)

# Load the Deep Classifier model
model_attention = torch.load(path_model_Attention)
norm_params_model_attention = np.load(path_model_Attention_params)

# Load the Charge Integration PSD parameters
CI_parameters = np.load(path_CI_parameters)

# Load the Normalized Cross-Correlation PSD parameters
Corr_parameters = np.load(path_NCC_classifier_parameters)

# Load the DNN with 6 output neurons: n,g,nn,ng,gn,gg
model_bm_nn = torch.load(path_model_BM_FCNN)
norm_params_model_bm_nn = np.load(path_model_BM_FCNN_params)

# Load the method utilizing two separate DNNs, the first for n,g and the second for nn,ng,gn,gg
model_bm_nn1 = torch.load(path_model_BM_FCNN1)
norm_params_model_bm_nn1 = np.load(path_model_BM_FCNN1_params)

# Performance of all methods are saved in statistics_mat
result_trig_iters = 30 # Number of iterations
statistics_mat = Classifier_Known_Arrivals_All_BM_Results(result_trig_iters,neutrons_test,gammas_test)